In [5]:
import os
os.chdir('/home/damon/Github/mmlab/mmdetection/')
import os.path as osp
import sys
import numpy as np
import torch, torchvision
import torch.nn.functional as F
from torch import nn
import matplotlib.pyplot as plt
import mmdet
import mmcv
%load_ext autoreload
%autoreload 2
from mmdet.models import build_detector
from mmdet.datasets import get_dataset
from mmdet.datasets import transforms

from visdrone.utils import box_ops
from time import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import tempfile
from mmdet.core.evaluation import coco_utils
from mmdet.core.post_processing.bbox_nms import multiclass_nms
from mmdet.ops.nms import nms_wrapper

In [7]:
cfg = 'visdrone/configs/ssd300.py'
cfg = mmcv.Config.fromfile(cfg)
# gt_json = cfg.data.test.ann_file
dataset = get_dataset(cfg.data.test)

loading annotations into memory...
Done (t=0.17s)
creating index...
index created!
load_annotations time: 0.2s from data/VisDrone2019-DET/VisDrone2018-DET-val/annotations_val.json


In [8]:
txt_dir = '/tmp/fuckyoudir/'

In [9]:
def txt2det(fid, num_classes=10):
    """ Returns detecton result from one txt(image)
    Args:
        fid: opened file handler.
    """
    lines = fid.readlines()
    lines = [v.strip('\n') for v in lines]
    lines = [v.split(',') for v in lines]
    dets = [[] for _ in range(num_classes)]
    for line in lines:
        x1, y1, w, h, sc, label, trun, occ = line
        label = int(label)
        if label == 0 or label == 11:
            continue
        assert label > 0 and label < 11, 'Bad label'
        x1, y1 = int(x1), int(y1)
        w, h = int(w), int(h)
        score = float(sc)
        x2 = x1 + w
        y2 = y1 + h
        bbox = np.asarray([x1, y1, x2, y2, score], dtype=np.float32)
        dets[label - 1].append(bbox)
    for i in range(len(dets)):
        if len(dets[i]) == 0:
            dets[i] = np.empty([0, 5])
        else:
            dets[i] = np.stack(dets[i], 0)
    return dets

In [10]:
def get_dets(savedir, dataset):
    """ This function return dets format on original image,
    ignoring crops, it calls txt2det(). 
    
    Returns:
        list(image) of list(class) of [N, 5]
    """
    results = []
    for info in dataset.img_infos:
        img_name = info['filename']
        stem = img_name.split('.')[0].split('/')[1]

        # do merge
        for i in range(10):
            if i == 9:
                exp_name = '{}_{}.txt'.format(stem, i)
                exp_name = os.path.join(txt_dir, exp_name)
                assert osp.exists(exp_name)
                with open(exp_name) as f:
                    result = txt2det(f)

        results.append(result)    
    return results

def get_dets_nine_merge(savedir, dataset):
    """ This function return dets format on original image,
    MERGE crops, it calls txt2det(). 
    
    Returns:
        list(image) of list(class) of [N, 5]
    """
    results = []
    for info in dataset.img_infos:
        img_name = info['filename']
        stem = img_name.split('.')[0].split('/')[1]

        # do merge
        single_results =  []
        for i in range(9):  # not 10
            exp_name = '{}_{}.txt'.format(stem, i)
            exp_name = os.path.join(txt_dir, exp_name)
            assert osp.exists(exp_name)
            with open(exp_name) as f:
                sin_res = txt2det(f)
            single_results.append(sin_res)
        
        # 
        num_classes = len(single_results[0])
        per_cls_bboxes = [[] for _ in range(num_classes)]  # [ []*10]
        for result in single_results:
            for i, res in enumerate(result):
                per_cls_bboxes[i].append(res)
        for i in range(num_classes):
            per_cls_bboxes[i] = np.concatenate(per_cls_bboxes[i], 0)
            
        results.append(per_cls_bboxes)    
    return results

In [11]:
def get_dets_merge(savedir, dataset):
    """ This function return dets format on original image,
    MERGE crops, it calls txt2det(). 
    
    Returns:
        list(image) of list(class) of [N, 5]
    """
    results = []
    for info in dataset.img_infos:
        img_name = info['filename']
        stem = img_name.split('.')[0].split('/')[1]

        # do merge
        single_results =  []
        for i in range(10):
            exp_name = '{}_{}.txt'.format(stem, i)
            exp_name = os.path.join(txt_dir, exp_name)
            assert osp.exists(exp_name)
            with open(exp_name) as f:
                sin_res = txt2det(f)
            single_results.append(sin_res)
        
        # 
        num_classes = len(single_results[0])
        per_cls_bboxes = [[] for _ in range(num_classes)]  # [ []*10]
        for result in single_results:
            for i, res in enumerate(result):
                per_cls_bboxes[i].append(res)
        for i in range(num_classes):
            per_cls_bboxes[i] = np.concatenate(per_cls_bboxes[i], 0)
            
        results.append(per_cls_bboxes)    
    return results

In [27]:
def transfrom_by_refine(results):
    for idx, res in enumerate(results):
        box_ops.refine_boxes_multi_class(res, 10, 0.5, 500, 0.5)

def transform_results_by_nms(results, nms_func, iou_thr = 0.5):
    for idx, res in enumerate(results):
        if idx% 50 == 0:
            print(idx)
        for i, c_res in enumerate(res):
#             bb, ind = nms_func(c_res.astype(np.float32), iou_thr)
            bb, ind = nms_func(c_res, iou_thr)
            res[i] = bb
    results[idx] = res
    return results

# Original

In [32]:
results = get_dets(savedir=txt_dir, dataset=dataset)
tf = tempfile.mkstemp(suffix='.json')
js = coco_utils.results2json(dataset, results, tf[1])
coco_utils.coco_eval(tf[1], ['bbox'], dataset.coco)

Loading and preparing results...
DONE (t=0.57s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=46.79s).
Accumulating evaluation results...
DONE (t=1.06s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.158
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.311
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.147
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.067
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.258
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.368
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.078
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.206
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.262
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

{'Precision/mAP': 0.1583621387131612,
 'Precision/mAP_.50IOU': 0.3114064199320829,
 'Precision/mAP_.75IOU': 0.14689942592288502,
 'Precision/mAP_small': 0.06689939720842975,
 'Precision/mAP_medium': 0.2576843857351151,
 'Precision/mAP_large': 0.36846566961701793,
 'Recall/AR_1': 0.07803980081232532,
 'Recall/AR_10': 0.20572231472956934,
 'Recall/AR_100': 0.26207509006712326,
 'Recall/AR_100_small': 0.14856367912645996,
 'Recall/AR_100_medium': 0.40767965796510275,
 'Recall/AR_100_large': 0.49493765152656577}

# Nine crop 19.9 (NMS) 18.2 (BV)

In [12]:
# results = get_dets(savedir=txt_dir, dataset=dataset)
results = get_dets_nine_merge(savedir=txt_dir, dataset=dataset)
tf = tempfile.mkstemp(suffix='.json')
js = coco_utils.results2json(dataset, results, tf[1])
coco_utils.coco_eval(tf[1], ['bbox'], dataset.coco)

In [17]:
results2 = results.copy()

a = time()
for idx, res in enumerate(results2):
    results2[idx] = box_ops.refine_boxes_multi_class(res, 10, 0.5, 500, 0.5, 'cuda')
print(time() - a)

In [19]:
tf = tempfile.mkstemp(suffix='.json')
js = coco_utils.results2json(dataset, results2, tf[1])
coco_utils.coco_eval(tf[1], ['bbox'], dataset.coco)

Loading and preparing results...
DONE (t=2.66s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=56.45s).
Accumulating evaluation results...
DONE (t=1.47s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.182
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.361
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.162
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.132
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.238
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.169
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.090
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.265
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.345
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

{'Precision/mAP': 0.18152895892003873,
 'Precision/mAP_.50IOU': 0.36118460621558557,
 'Precision/mAP_.75IOU': 0.16235401442423017,
 'Precision/mAP_small': 0.1322194113245694,
 'Precision/mAP_medium': 0.2376305321307981,
 'Precision/mAP_large': 0.16890914307765736,
 'Recall/AR_1': 0.08964318150074709,
 'Recall/AR_10': 0.2648194551138418,
 'Recall/AR_100': 0.34507663813984446,
 'Recall/AR_100_small': 0.286048156720386,
 'Recall/AR_100_medium': 0.40561617434632735,
 'Recall/AR_100_large': 0.25026917021271133}

In [25]:
import functools
nms_func = functools.partial(nms_wrapper.nms)
ress = transform_results_by_nms(results, nms_func)

tf = tempfile.mkstemp(suffix='.json')
js = coco_utils.results2json(dataset, ress, tf[1])
coco_utils.coco_eval(tf[1], ['bbox'], dataset.coco)

0
50
100
150
200
250
300
350
400
450
500
Loading and preparing results...
DONE (t=6.10s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=57.52s).
Accumulating evaluation results...
DONE (t=1.52s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.199
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.396
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.177
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.144
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.272
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.201
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.090
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.273
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.370
 Average Recall     (AR) @

{'Precision/mAP': 0.19934789162412905,
 'Precision/mAP_.50IOU': 0.39584313814755995,
 'Precision/mAP_.75IOU': 0.17698003778766463,
 'Precision/mAP_small': 0.143585849038906,
 'Precision/mAP_medium': 0.27173026345722057,
 'Precision/mAP_large': 0.20086040007452463,
 'Recall/AR_1': 0.08989366138529731,
 'Recall/AR_10': 0.27323734467455346,
 'Recall/AR_100': 0.3703359690489746,
 'Recall/AR_100_small': 0.30546274071794377,
 'Recall/AR_100_medium': 0.44968138713575617,
 'Recall/AR_100_large': 0.28537364669719056}

# 10 crop

In [28]:
results = get_dets_merge(savedir=txt_dir, dataset=dataset)

tf = tempfile.mkstemp(suffix='.json')
js = coco_utils.results2json(dataset, results, tf[1])
coco_utils.coco_eval(tf[1], ['bbox'], dataset.coco)

Loading and preparing results...
DONE (t=9.52s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=65.23s).
Accumulating evaluation results...
DONE (t=1.92s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.112
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.201
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.109
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.077
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.163
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.267
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.095
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.245
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.417
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

{'Precision/mAP': 0.11220081274237778,
 'Precision/mAP_.50IOU': 0.20088392625434084,
 'Precision/mAP_.75IOU': 0.1085958834019823,
 'Precision/mAP_small': 0.07723474704863671,
 'Precision/mAP_medium': 0.1630736799805062,
 'Precision/mAP_large': 0.26690191022460164,
 'Recall/AR_1': 0.09473657483109706,
 'Recall/AR_10': 0.24486847911850504,
 'Recall/AR_100': 0.416658497362242,
 'Recall/AR_100_small': 0.3295318452536646,
 'Recall/AR_100_medium': 0.5223041431155542,
 'Recall/AR_100_large': 0.5131086261724264}

In [29]:
results2 = results.copy()

a = time()
for idx, res in enumerate(results2):
    results2[idx] = box_ops.refine_boxes_multi_class(res, 10, 0.5, 500, 0.5, 'cuda')
print(time() - a)

79.14045810699463


In [30]:
tf = tempfile.mkstemp(suffix='.json')
js = coco_utils.results2json(dataset, results2, tf[1])
coco_utils.coco_eval(tf[1], ['bbox'], dataset.coco)

Loading and preparing results...
DONE (t=2.62s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=59.48s).
Accumulating evaluation results...
DONE (t=1.55s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.183
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.353
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.170
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.127
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.243
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.228
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.092
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.272
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.357
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

{'Precision/mAP': 0.18304350687001456,
 'Precision/mAP_.50IOU': 0.35331328503357784,
 'Precision/mAP_.75IOU': 0.17034149981582647,
 'Precision/mAP_small': 0.12705158205697883,
 'Precision/mAP_medium': 0.24295604260602963,
 'Precision/mAP_large': 0.22790424703683193,
 'Recall/AR_1': 0.09235347186580027,
 'Recall/AR_10': 0.2720781807554025,
 'Recall/AR_100': 0.3567038433928441,
 'Recall/AR_100_small': 0.28946884417197705,
 'Recall/AR_100_medium': 0.4283635875455585,
 'Recall/AR_100_large': 0.376045742378444}

In [31]:
import functools
nms_func = functools.partial(nms_wrapper.nms)
ress = transform_results_by_nms(results, nms_func)

tf = tempfile.mkstemp(suffix='.json')
js = coco_utils.results2json(dataset, ress, tf[1])
coco_utils.coco_eval(tf[1], ['bbox'], dataset.coco)

0
50
100
150
200
250
300
350
400
450
500
Loading and preparing results...
DONE (t=6.86s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=63.56s).
Accumulating evaluation results...
DONE (t=1.69s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.213
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.412
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.193
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.143
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.295
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.347
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.095
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.282
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.384
 Average Recall     (AR) @

{'Precision/mAP': 0.21313772651642726,
 'Precision/mAP_.50IOU': 0.41238686011696435,
 'Precision/mAP_.75IOU': 0.1932186954267904,
 'Precision/mAP_small': 0.14309864894602342,
 'Precision/mAP_medium': 0.29454960950873404,
 'Precision/mAP_large': 0.34678927472224863,
 'Recall/AR_1': 0.09473586379582971,
 'Recall/AR_10': 0.2824933560135721,
 'Recall/AR_100': 0.38428789856128254,
 'Recall/AR_100_small': 0.308652821923981,
 'Recall/AR_100_medium': 0.47575187036337424,
 'Recall/AR_100_large': 0.4841151369574703}